# Bidding

This is a tutorial for how to use the bidding engine

In [ ]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

In [ ]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
models.search_threshold = 0.07
models.double_dummy_eval = True
# Filter to remove hands not consistent with the bidding. Used during play
models.bid_accept_play_threshold = 0.01
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies
#sampler.use_distance = False
#sampler.bidding_threshold_sampling = 0.25


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [ ]:

hands = ["AJ.KJ9.A9764.AJ8","T853.AT65.Q8.T52", "K976.Q874.KT.K74", "Q42.32.J532.Q963"]
#hands = ['AKQJTx.AQTx.Ax.x', 'xxx.Kx.KT8x.KJxx', 'x.J98xxxx.9.QTxx', '98x..QJxxxx.A98x']

In [ ]:
# North is dealer
dealer = 0
bidder_bots = [BotBid([False, True], hand, models, sampler, i, dealer, False) for i, hand in enumerate(hands)]

In [ ]:
auction = [] 

turn_i = 0 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    #if step== 5:
    #     bid.bid='X'
    #if step== 1:
    #     bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)